In [1]:
import pandas as pd
import httpx

In [2]:
def fetch_mlb_standings():
    url = "https://site.api.espn.com/apis/site/v2/sports/baseball/mlb/standings"
    response = httpx.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

In [3]:
fetch_mlb_standings()

{'fullViewLink': {'text': 'Full Standings',
  'href': 'https://www.espn.com/mlb/standings'}}